In [1]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Input, Flatten, Dropout
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import train_test_split
from load_image import *

def model_VGG16(params):
    # Load the VGG16 model
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

    # Freeze all layers
    for layer in vgg16.layers[:int(params['vgg_layer'])]:
        layer.trainable = False

    # Build new classifier
    x = vgg16.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(rate=params['dropout'])(x)
    x = Dense(len(class_names), activation='softmax')(x)

    # Create new model
    model = Model(inputs=vgg16.input, outputs=x)
    model.compile(loss="categorical_crossentropy",
                  optimizer=Adam(learning_rate=0.0001),
                  metrics=["accuracy"])
    
    return model

def objectives(params):
    model = model_VGG16(params)
    model.fit(train_data_augmented,
              epochs=30,
              batch_size=params['batch_size'],
              validation_data=validation_data)
    
    score, acc = model.evaluate(validation_data,
                                batch_size=params['batch_size'])
    
    return {
        'loss': -acc,
        'status': STATUS_OK
    }
    

space = {
    'vgg_layer': hp.quniform('vgg_layer', 10, 14, 1),
    # 'learning_rate': hp.uniform('learning_rate', 0.0001, 0.1),
    'dropout': hp.uniform('dropout', 0.0, 0.5),
    'batch_size': hp.quniform('batch_size', 32, 64, 32),
    # 'epochs': hp.quniform('epochs', 30, 60, 30)
}

trials = Trials()
best = fmin(objectives, space, algo=tpe.suggest, max_evals=1, verbose=1, trials=trials)
print(f'Best results : {best}')

Found 714 images belonging to 7 classes.
Found 112 images belonging to 7 classes.
Found 714 images belonging to 7 classes.
Epoch 1/30                                             

90/90 [==============================] - 297s 3s/step - loss: 0.5064 - accuracy: 0.8291 - val_loss: 1.2804 - val_accuracy: 0.7232

Epoch 2/30                                             

90/90 [==============================] - 313s 3s/step - loss: 0.0454 - accuracy: 0.9832 - val_loss: 1.4330 - val_accuracy: 0.6875

Epoch 3/30                                             

90/90 [==============================] - 319s 4s/step - loss: 0.1025 - accuracy: 0.9678 - val_loss: 0.6454 - val_accuracy: 0.7857

Epoch 4/30                                             

90/90 [==============================] - 328s 4s/step - loss: 0.0914 - accuracy: 0.9706 - val_loss: 0.8198 - val_accuracy: 0.6696

Epoch 5/30                                             

90/90 [==============================] - 323s 4s/step - loss: 0.0631

KeyboardInterrupt: 